# Logistic Regression: Regularized and Nonregularized


Hello, welcome to this notebook. Today, I'll try to implement a logistic regression model from scratch with (Lasso,L2)and without regularization and see the difference.

Let's start


### Table Of Contents

1. [Loading and Examining Data](#1)
2. [Defining Sigmoid Function](#2)
3. [Defining Cost Functions](#3)
4. [Gradient Descent](#4)
5. [Evaluating Models](#5)
6. [Results](#6)

<a id=1></a>
## 1. Loading and Examining Data

We'll start with loading data and examining it to get our first insights. Dataset can be found on [Kaggle](https://www.kaggle.com/datasets/alphiree/cardiovascular-diseases-risk-prediction-dataset)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

data = pd.read_csv("CVD_cleaned.csv")

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308854 entries, 0 to 308853
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   General_Health                308854 non-null  object 
 1   Checkup                       308854 non-null  object 
 2   Exercise                      308854 non-null  object 
 3   Heart_Disease                 308854 non-null  object 
 4   Skin_Cancer                   308854 non-null  object 
 5   Other_Cancer                  308854 non-null  object 
 6   Depression                    308854 non-null  object 
 7   Diabetes                      308854 non-null  object 
 8   Arthritis                     308854 non-null  object 
 9   Sex                           308854 non-null  object 
 10  Age_Category                  308854 non-null  object 
 11  Height_(cm)                   308854 non-null  float64
 12  Weight_(kg)                   308854 non-nul

In [2]:
data.head()

,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,Poor,Within the past 2 years,No,No,No,No,No,No,Yes,Female,70-74,150.0,32.66,14.54,Yes,0.0,30.0,16.0,12.0
1,Very Good,Within the past year,No,Yes,No,No,No,Yes,No,Female,70-74,165.0,77.11,28.29,No,0.0,30.0,0.0,4.0
2,Very Good,Within the past year,Yes,No,No,No,No,Yes,No,Female,60-64,163.0,88.45,33.47,No,4.0,12.0,3.0,16.0
3,Poor,Within the past year,Yes,Yes,No,No,No,Yes,No,Male,75-79,180.0,93.44,28.73,No,0.0,30.0,30.0,8.0
4,Good,Within the past year,No,No,No,No,No,No,No,Male,80+,191.0,88.45,24.37,Yes,0.0,8.0,4.0,0.0


##### Heart disease is our target feature

* Let's start by processing string features. 

In [3]:
data.General_Health.unique()

array(['Poor', 'Very Good', 'Good', 'Fair', 'Excellent'], dtype=object)

In [4]:
# We should one hot encode this categorical feature

gen_health_encoded = pd.get_dummies(data["General_Health"],prefix="gen_health_")
gen_health_encoded.head()

,gen_health__Excellent,gen_health__Fair,gen_health__Good,gen_health__Poor,gen_health__Very Good
0,0,0,0,1,0
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,0,1,0
4,0,0,1,0,0


In [5]:
data.Checkup.unique()

array(['Within the past 2 years', 'Within the past year',
       '5 or more years ago', 'Within the past 5 years', 'Never'],
      dtype=object)

In [6]:
checkup_encoded = pd.get_dummies(data["Checkup"],prefix="checkup_")
checkup_encoded.head()

,checkup__5 or more years ago,checkup__Never,checkup__Within the past 2 years,checkup__Within the past 5 years,checkup__Within the past year
0,0,0,1,0,0
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1


* As we see; Exercise,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Smoking_History features are binary features and we can get their dummies too.

In [7]:
dummy_features = "Exercise,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Smoking_History".split(",")
dummy_dfs = []
for f in dummy_features:
    
    dummy_df_f = pd.get_dummies(data[f],prefix=f+"_")
    dummy_dfs.append(dummy_df_f)
    

binary_features_encoded = pd.concat(dummy_dfs,axis=1)

binary_features_encoded.head()

,Exercise__No,Exercise__Yes,Skin_Cancer__No,Skin_Cancer__Yes,Other_Cancer__No,Other_Cancer__Yes,Depression__No,Depression__Yes,Diabetes__No,"Diabetes__No, pre-diabetes or borderline diabetes",Diabetes__Yes,"Diabetes__Yes, but female told only during pregnancy",Arthritis__No,Arthritis__Yes,Sex__Female,Sex__Male,Smoking_History__No,Smoking_History__Yes
0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,1,0,0,1
1,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0
2,0,1,1,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0
3,0,1,1,0,1,0,1,0,0,0,1,0,1,0,0,1,1,0
4,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,1,0,1


* And the last categorical feature we have is Age_Category

In [8]:
data["Age_Category"].unique()

array(['70-74', '60-64', '75-79', '80+', '65-69', '50-54', '45-49',
       '18-24', '30-34', '55-59', '35-39', '40-44', '25-29'], dtype=object)

In [9]:
age_category_encoded = pd.get_dummies(data["Age_Category"],prefix="Age_Category_")
age_category_encoded.head()

,Age_Category__18-24,Age_Category__25-29,Age_Category__30-34,Age_Category__35-39,Age_Category__40-44,Age_Category__45-49,Age_Category__50-54,Age_Category__55-59,Age_Category__60-64,Age_Category__65-69,Age_Category__70-74,Age_Category__75-79,Age_Category__80+
0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1


In [10]:
# Now we can concatenate all categorical features and go back to the numerical features

encoded_features_concat = pd.concat([gen_health_encoded,
                                     checkup_encoded,
                                     binary_features_encoded,
                                     age_category_encoded],axis=1)


encoded_features_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308854 entries, 0 to 308853
Data columns (total 41 columns):
 #   Column                                                Non-Null Count   Dtype
---  ------                                                --------------   -----
 0   gen_health__Excellent                                 308854 non-null  uint8
 1   gen_health__Fair                                      308854 non-null  uint8
 2   gen_health__Good                                      308854 non-null  uint8
 3   gen_health__Poor                                      308854 non-null  uint8
 4   gen_health__Very Good                                 308854 non-null  uint8
 5   checkup__5 or more years ago                          308854 non-null  uint8
 6   checkup__Never                                        308854 non-null  uint8
 7   checkup__Within the past 2 years                      308854 non-null  uint8
 8   checkup__Within the past 5 years                      308854 non

In [11]:
numerical_features = ["Height_(cm)",
                      "Weight_(kg)",
                      "BMI","Alcohol_Consumption",
                      "Fruit_Consumption",
                      "Green_Vegetables_Consumption",
                      "FriedPotato_Consumption"]


* Normalizing numerical features is the best practice!

In [12]:
def normalize(x):
    
    """
    Performs Z-score normalization on given x. 
    Args:
    x (ndarray) (m,): m values
    Returns:
    normalized_x (ndarray) (m,): normalized values
    """
    
    mean = np.mean(x)
    std = np.std(x)
    
    normalized_x = (x - mean) / std
    
    return normalized_x

* In general, we should store mean and standart deviation values to be able to perform normalization after training. **Because non-normalized examples have to be normalized** if we want our model to predict correct, and they should be normalized with the parameters of traning set.
<br>

* However since this notebook is just for educational purposes and we won't use the model with any new example, we can ignore that.

In [13]:
normalized_numerical_features = []
for f in numerical_features:
    
    normalized_f = pd.Series( normalize(data[f].to_numpy() ),name=f+"_normalized")
    normalized_numerical_features.append(normalized_f)


norm_numerical_concat = pd.concat(normalized_numerical_features,axis=1)
norm_numerical_concat.info()
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308854 entries, 0 to 308853
Data columns (total 7 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   Height_(cm)_normalized                   308854 non-null  float64
 1   Weight_(kg)_normalized                   308854 non-null  float64
 2   BMI_normalized                           308854 non-null  float64
 3   Alcohol_Consumption_normalized           308854 non-null  float64
 4   Fruit_Consumption_normalized             308854 non-null  float64
 5   Green_Vegetables_Consumption_normalized  308854 non-null  float64
 6   FriedPotato_Consumption_normalized       308854 non-null  float64
dtypes: float64(7)
memory usage: 16.5 MB


In [14]:
norm_numerical_concat.head()

,Height_(cm)_normalized,Weight_(kg)_normalized,BMI_normalized,Alcohol_Consumption_normalized,Fruit_Consumption_normalized,Green_Vegetables_Consumption_normalized,FriedPotato_Consumption_normalized
0,-1.934250,-2.386180,-2.159696,-0.621527,0.006625,0.059597,0.664502
1,-0.526857,-0.303547,-0.051548,-0.621527,0.006625,-1.012342,-0.267579
2,-0.714510,0.227770,0.742649,-0.133707,-0.716973,-0.811354,1.130543
3,0.880535,0.461569,0.015913,-0.621527,0.006625,0.997544,0.198462
4,1.912623,0.227770,-0.652562,-0.621527,-0.877772,-0.744358,-0.733620


* Our input features ready, now let's convert our target feature to integer and we are done with pandas!

In [15]:
target_feature = [1 if v=="Yes" else 0 for v in data["Heart_Disease"] ]
print(target_feature[:5])

[0, 1, 0, 1, 0]


In [16]:
X = pd.concat([encoded_features_concat,norm_numerical_concat],axis=1)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308854 entries, 0 to 308853
Data columns (total 48 columns):
 #   Column                                                Non-Null Count   Dtype  
---  ------                                                --------------   -----  
 0   gen_health__Excellent                                 308854 non-null  uint8  
 1   gen_health__Fair                                      308854 non-null  uint8  
 2   gen_health__Good                                      308854 non-null  uint8  
 3   gen_health__Poor                                      308854 non-null  uint8  
 4   gen_health__Very Good                                 308854 non-null  uint8  
 5   checkup__5 or more years ago                          308854 non-null  uint8  
 6   checkup__Never                                        308854 non-null  uint8  
 7   checkup__Within the past 2 years                      308854 non-null  uint8  
 8   checkup__Within the past 5 years            

In [17]:
X = X.to_numpy()
y = np.asarray(target_feature)

print("Shape of X",X.shape)
print("Shape of y",y.shape)

Shape of X (308854, 48)
Shape of y (308854,)


* Now we can do train test splitting and our data will be ready!

In [18]:
def train_test_split(X,y,test_size=0.2,random_state=42):
    
    """
    Splits the dataset as training and testing sets
    Args:
    X (ndarray)(m,n), y (ndarray)(m,): input and target features
    test_size (scalar) = ratio of the test set to the entire dataset.
    random_state (scalar) = sets the seed to get the same split everytime
    
    Returns:
    X_train,X_test,y_train,y_test
    """
    
    np.random.seed(random_state)
    
    m = X.shape[0]
    size_of_test_set = int(m * test_size)
    
    # We'll shuffle indices, the first size_of_test_set will be test set, the rest will be training set.
    shuffled_indices = np.random.permutation(np.arange(m))
    test_indices = shuffled_indices[:size_of_test_set]
    train_indices = shuffled_indices[size_of_test_set:]
    
    
    X_train = X[train_indices]
    X_test = X[test_indices]
    y_train = y[train_indices]
    y_test = y[test_indices]
    
    return X_train,X_test,y_train,y_test
    

In [19]:
X_train,X_test,y_train,y_test = train_test_split(X,y)

print("Shape of X_train",X_train.shape)
print("Shape of X_test",X_test.shape)
print("Shape of y_train",y_train.shape)
print("Shape of y_test",y_test.shape)

Shape of X_train (247084, 48)
Shape of X_test (61770, 48)
Shape of y_train (247084,)
Shape of y_test (61770,)


<a id=2></a>
## 2. Defining Sigmoid Function

Sigmoid function is the heart of Logistic Regression, it's a special function which ranges from 0 to 1 and it's value is 0.5 at 0. 

We can interpret g(z) as the probability of the given input being positive. Such as 0.7 means the label is %70 chance positive. 

It's formula is:

$$g(z) = {1 \over 1+e^-z} $$


In [20]:
def sigmoid(z):
    
    g = 1 / (1 + np.exp(-z))
    return g

<a id=3></a>
## 3. Defining Cost Functions

We've prepared the data and now we can define cost functions for the logistic regression model. The first function we'll define is logistic regression cost function:


<h3>$$L(f_w,_b(X^i),y^i) = -y^ilog(f_w,_b(X^i)) - (1-y^i)log(1-f_w,_b(X^i))$$</h3>
<h3>$$J(w,b) = {1 \over m} \sum_{i=1}^{m} L(f_w,_b(X^i),y^i)  $$ </h3>

Other one is logistic regression cost function with regularization term. Definition of the regularization term is:

<h3>$$ {\lambda \over m} \sum_{j=1}^{n}w_j^2 $$</h3>

So the latter is: 

<h3>$$J(w,b) = {1 \over m} \sum_{i=1}^{m} L(f_w,_b(X^i),y^i) + {\lambda \over m} \sum_{j=1}^{n}w_j^2$$</h3>

* But before coding these functions, let's talk about how regularization works. 

* Our main goal in logistic regression (just like most other algorithms) is minimizing the cost. When we add $w_j^2$ to our cost function, we **penalize** very big positive and negative weights. Since gradient descent minimizes the cost, our weights will shrink a little bit each iteration.

* And this way we'll get a simpler model with lower variance since our model is less complex.

In [21]:
def compute_logloss(X,y,w,b):
    
    z = np.dot(X,w) + b
    f_wb = sigmoid(z)
    
    loss = -y*np.log(f_wb) -(1-y)*np.log(1-f_wb)
    return loss

In [22]:
def compute_cost(X,y,w,b):
    
    m = X.shape[0]
    
    loss = compute_logloss(X,y,w,b)
    cost = np.sum(loss) / m
    
    return cost

In [23]:
def compute_cost_reg(X,y,w,b,lambda_):
    
    m = X.shape[0]
    cost = compute_cost(X,y,w,b)
    
    w_squared = w**2
    w_squared_sum = np.sum(w_squared)
    
    reg_term = w_squared_sum / m
    reg_term = lambda_ * reg_term
    
    return cost + reg_term

<a id=4></a>
## 4. Gradient Descent

Since we use the partial derivatives of $J(w,b)$ to update the model, derivating them might be useful later:

* Derivative of J(w,b) with respect to the $w_j$ without regularization term:

<h3>$$ {d \over dw_j} J(w,b) =  {1 \over m} \sum_{i=1}^{m} (f_w,_b(x^i)-y^i)x^i$$</h3>

* Derivative of J(w,b) with respect to the $b$ without regularization term:

<h3>$$ {d \over db} J(w,b) =  {1 \over m} \sum_{i=1}^{m} (f_w,_b(x^i)-y^i)$$</h3>

* Derivative of J(w,b) with respect to the $w_j$ **with** regularization term:

<h3>$$ {d \over dw_j} J(w,b) =  {1 \over m} \sum_{i=1}^{m} (f_w,_b(x^i)-y^i)x^i + {\lambda \over m} w_j$$</h3>


* Derivative of $J(w,b)$ with respect to the $b$ **doesn't change** since we don't apply regularization to $b$ parameter.


In [24]:
def compute_gradients(X,y,w,b):
    
    """
    Computes gradients of parameters
    """
    
    m,n = X.shape
    
    dj_dw = np.zeros((n,))
    dj_db = 0.0
    
    z = np.dot(X,w) + b
    f_wb = sigmoid(z)
    err = f_wb - y
    
    for j in range(n):
        dj_dw_j = err * X[:,j]
        dj_dw_j = np.sum(dj_dw_j) / m
        dj_dw[j] = dj_dw_j
    
    dj_db = np.sum(err) / m
    
    return dj_dw,dj_db

In [25]:
def compute_gradients_reg(X,y,w,b,lambda_):
    
    m,n = X.shape
    
    dj_dw,dj_db = compute_gradients(X,y,w,b)
    
    for j in range(n):
        reg_term_grad = (lambda_ / m) * w[j]
        dj_dw[j] = dj_dw[j] + reg_term_grad
        
    
    return dj_dw,dj_db

* Now we have the functions to compute gradients of parameters, we can define a function to perform gradient descent:

In [26]:
def gradient_descent(X,y,w_init,b_init,learning_rate,num_iters,lambda_):
    
    """
    Trains the model with batch gradient descent
    Args:
    X (ndarray), y (ndarray) input features, target label
    w_init (ndarray), b_init (scalar) initialization parameters of the model
    learning_rate (int or float) scales the step 
    num_iters (int) number of iterations to perform gradient descent
    lambda_ (int or float) scales the regularization
    
    Returns:
    w (ndarray),b (ndarray) updated parameters of the model
    J_history (list) cost history
    """
    
    w = copy.deepcopy(w_init)
    b = copy.deepcopy(b_init)
    J_history = []
    
    for i in range(num_iters):
        
        cost_i = compute_cost_reg(X,y,w,b,lambda_)
        dj_dw,dj_db = compute_gradients_reg(X,y,w,b,lambda_)
    
        w = w - learning_rate*dj_dw
        b = b - learning_rate*dj_db
        
        if (i+1)%10 == 0:
            J_history.append(cost_i)
            
        
        if (i+1)%200 == 0:
            print("Number of iterations {}, cost: {}".format((i+1),cost_i))
    
    return w,b,J_history

* Let's initialize our parameters as zeros and update them.

In [27]:
# First we'll set lambda_ to 0 and update model without regularization
m,n = X.shape

w_init = np.zeros((n,))
b_init = 0.0

cost_init = compute_cost_reg(X_train,y_train,w_init,b_init,0)
print("Cost at initialization: {}".format(cost_init))

w,b,J_history = gradient_descent(X=X_train,
                                 y=y_train,
                                 w_init=w_init,
                                 b_init=b_init,
                                 learning_rate=0.01,
                                 num_iters=1000,
                                 lambda_=0
                                )


Cost at initialization: 0.693147180559945
Number of iterations 200, cost: 0.27557942194114277
Number of iterations 400, cost: 0.2628233496169082
Number of iterations 600, cost: 0.2587245009299135
Number of iterations 800, cost: 0.25584585267147875
Number of iterations 1000, cost: 0.2534665188535786


In [28]:
# Now let's set lambda_ to something random and do with regularization

cost_init = compute_cost_reg(X_train,y_train,w_init,b_init,2)
print("With Regularization | Cost at initialization: {}".format(cost_init))

w_reg,b_reg,J_history_reg = gradient_descent(X=X_train,
                                 y=y_train,
                                 w_init=w_init,
                                 b_init=b_init,
                                 learning_rate=0.01,
                                 num_iters=1000,
                                 lambda_=5
                                )

With Regularization | Cost at initialization: 0.693147180559945
Number of iterations 200, cost: 0.2755905922625693
Number of iterations 400, cost: 0.26283844639422005
Number of iterations 600, cost: 0.2587416875533887
Number of iterations 800, cost: 0.2558647655207219
Number of iterations 1000, cost: 0.2534871184331508


<a id=5></a>
## 5. Evaluating Models

Let's compare our models, for this we'll use accuracy and check the parameters.

In [29]:
def predict(X,w,b,threshold=0.5):
    """
    Does prediction using trained model
    Args:
    X (ndarray) input features
    w (ndarray),b (scalar) parameters of the model
    threshold (float): a value between 0 and 1, determines which values are positive and negative
    
    Returns:
    y_pred (ndarray) predictions
    """
    
    z = np.dot(X,w) + b
    f_wb = sigmoid(z)
    
    y_pred = (f_wb >= threshold).astype("int")
    return y_pred
    

In [30]:
# Let's use the test set
y_pred = predict(X_test,w,b)
print(y_pred[:5])

y_pred_reg = predict(X_test,w_reg,b_reg)
print(y_pred_reg[:5])

[0 0 0 0 0]
[0 0 0 0 0]


In [31]:
def accuracy(y,y_pred):
    """
    Computes accuracy
    Args:
    y: target label
    y_pred: prediction
    Returns:
    acc: accuracy in percent
    """
    
    m = len(y)
    correct_preds = np.sum((y == y_pred).astype("int"))
    
    acc = (correct_preds / m) * 100
    acc = round(acc,2)
    return acc

In [32]:
print("Accuracy of model on test set without regularization: {}%".format(accuracy(y_test,y_pred)))
print("Accuracy of model on test set with regularization {}%".format(accuracy(y_test,y_pred_reg)))


Accuracy of model on test set without regularization: 91.91%
Accuracy of model on test set with regularization 91.91%


In [33]:
print(w)

[-0.1415808   0.05207069 -0.08619781  0.05605094 -0.20631705 -0.03764439
 -0.00358803 -0.08772197 -0.04782913 -0.14919052 -0.02378703 -0.30218701
 -0.33287963  0.00690559 -0.330858    0.00488397 -0.25608281 -0.06989123
 -0.37895763 -0.00648187  0.06713654 -0.00767108 -0.33814668  0.01217264
 -0.25296533 -0.07300871 -0.2831656  -0.04280844 -0.0545514  -0.04621996
 -0.053243   -0.05892731 -0.05864449 -0.05108141 -0.05235987 -0.03851576
 -0.02596627 -0.0051878   0.02133848  0.03384504  0.06353972  0.0282224
  0.03405621  0.02126094 -0.05429783 -0.01390043 -0.0195298  -0.0215621 ]


In [34]:
print(w_reg)

[-0.14156894  0.05206367 -0.08619382  0.05604478 -0.20630115 -0.03764122
 -0.00358774 -0.0877148  -0.0478251  -0.14918659 -0.02378798 -0.30216748
 -0.3328589   0.00690345 -0.33083751  0.00488205 -0.25606835 -0.06988711
 -0.37893133 -0.00648156  0.06712787 -0.00767044 -0.33812183  0.01216637
 -0.25294701 -0.07300844 -0.28314519 -0.04281026 -0.05454667 -0.04621597
 -0.05323843 -0.05892226 -0.05863953 -0.05107719 -0.05235573 -0.03851317
 -0.02596519 -0.00518889  0.02133479  0.03384061  0.06353218  0.02821935
  0.03405265  0.02125875 -0.05429221 -0.01389886 -0.01952765 -0.02155985]


<a id=6></a>
## 6. Results

We didn't see big difference between the models when we computed their accuracy on the test set. Main reason we didn't have any problems with the unregularized model is it didn't **overfit** and therefore variance was low. 

However, if;
- our data had polynomial features (like $height^2$) 
- we had less data (we had 300k examples which was enough to generalise for this case.)
- we had more features

Regularizing could make a huge difference. 

And also when we check the parameters we can see that our regularization worked and shrank the parameters a little bit. 


Thanks for going through this notebook, I wanted to implement the logistic regression from scratch with regularization to better understand it. I hope it helped you too!